# BertTopic
In diesem Notebook trainieren wir die BERTTopic Modelle

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../')) ## needed to import the function.py file

from functions import *
import pandas as pd
from bertopic import BERTopic
from spacy.lang.de.stop_words import STOP_WORDS

2023-03-15 08:46:55.350962: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Config

In [2]:
## Import dataframe
filelocation = '../../data/DataClean2023'
df = pd.read_feather(filelocation)

# import custom stopwords list
customstopwords = pd.read_excel('../../config/customstopwords.xlsx')
customstopwords = customstopwords['stopword'].tolist()

customstopwords.extend(STOP_WORDS) # also add spacy stopwords

# Also add ortsnamen to the stoplist because we have them in the metadata and dont want them in the comments
orte = [x.lower() for x in set(df.ft_startort.tolist()) if x == x and x.lower() != '']

# Create the list of locations
for location in df.ft_startort.tolist():
    # Check if the value is a string
    if isinstance(location, str):
        # Convert to lowercase and remove 'Zug'
        location = location.lower()
        if location == 'zug':
            continue
        
        # Split the location into tokens if it contains whitespace
        tokens = location.split()
        
        # Add each token to the list individually
        for token in tokens:
            # Skip any token that is in the stoplist
            if token in orte:
                continue
            # Remove any commas from the end of the token
            token = token.rstrip(',')
            orte.append(token)
    
# Remove duplicates from the list
orte = list(set(orte))

orte.remove("zug")

# extend the stopword list with the ortsnamen
customstopwords.extend(orte)
customstopwords = list(dict.fromkeys(customstopwords)) #remove potenzial duplicates

# Daten Laden

In [18]:
## Import dataframe
filelocation = '../../data/DataText2023'
data = pd.read_feather(filelocation)

# Training 

In [28]:
docs = data["Kommentar"].to_list()

In [32]:
# Global Parameters for all models
min_topic_size=int(len(docs)*0.005) #dynamic topic size based on count of comments
stop_words=customstopwords
modelpath = "../../models/"

In [33]:
# Train Model 3: High Speed / Small Size / Lower Benchmark Scores
modelname = 'paraphrase-multilingual-MiniLM-L12-v2'
additional_tag = '_Trends_Q1_2023'
fit_berttopic_if_not_exists(modelpath+"BERTTopic_" + modelname + additional_tag +".model",docs=docs,embedding_model=modelname, min_topic_size=min_topic_size,stop_words=stop_words)